# Scraping Facebook


In [ ]:
# Imports et Configuration
import time
import random
import csv
from datetime import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import os

# Configuration globale
EMAIL = os.getenv('EMAIL') # À changer pour la sécurité
PASSWORD = os.getenv('PASSWORD')  # À changer pour la sécurité 


# Liste des profils à scraper - AJOUTEZ VOS URLS ICI
PROFILE_URLS = [
    "https://www.facebook.com/uvburkina",
    # Ajoutez d'autres URLs ici :
    # "https://www.facebook.com/autre_profil",
    # "https://www.facebook.com/encore_un_profil",
]

# Clés pour rechercher les postes par mot-clés
KEYWORDS = []

DEFAULT_PROFILE_URL = "https://www.facebook.com/uvburkina"
OUTPUT_FOLDER = "facebook_data"
SCROLL_STEP = 500
SCROLL_DELAY = 2

# Créer le dossier de sortie si nécessaire
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

print(" Configuration chargée")
print(f" Dossier de sortie: {OUTPUT_FOLDER}")
print(f" {len(PROFILE_URLS)} profil(s) configuré(s)")

In [ ]:
# Fonctions d'initialisation du driver
def initialize_driver():
    """Initialise le driver Chrome avec options anti-détection"""
    try:
        options = webdriver.ChromeOptions()
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--no-sandbox")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option("useAutomationExtension", False)
        
        driver = webdriver.Chrome(options=options)
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        
        print(" Driver initialisé avec succès")
        return driver
        
    except Exception as e:
        print(f" Erreur lors de l'initialisation du driver: {e}")
        return None

def close_driver(driver):
    """Ferme le driver de manière sécurisée"""
    if driver:
        try:
            driver.quit()
            print(" Driver fermé avec succès")
        except Exception as e:
            print(f" Erreur lors de la fermeture du driver: {e}")

In [ ]:
#  Fonctions de simulation humaine
def simulate_human_typing(element, text):
    """Simule une frappe humaine réaliste"""
    for char in text:
        element.send_keys(char)
        time.sleep(random.uniform(0.1, 0.3))
        if random.random() < 0.1:  # Pause aléatoire 10% du temps
            time.sleep(random.uniform(0.3, 0.7))

def slow_scroll(driver, step=SCROLL_STEP):
    """Fait défiler la page lentement"""
    driver.execute_script(f"window.scrollBy(0, {step});")
    time.sleep(SCROLL_DELAY)
    print(f" Scroll de {step}px effectué")

In [ ]:
# Fonctions de connexion
def login_to_facebook(driver, email=EMAIL, password=PASSWORD):
    """Se connecte à Facebook"""
    try:
        driver.get("https://www.facebook.com/login")
        print(" Accès à la page de connexion Facebook")
        
        # Attendre et saisir l'email
        email_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "email"))
        )
        simulate_human_typing(email_input, email)
        print(" Email saisi")
        
        # Attendre et saisir le mot de passe
        password_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "pass"))
        )
        simulate_human_typing(password_input, password)
        print(" Mot de passe saisi")
        
        # Cliquer sur le bouton de connexion
        login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
        ActionChains(driver)\
            .move_to_element(login_button)\
            .pause(random.uniform(0.2, 0.4))\
            .click()\
            .perform()
        
        print(" Tentative de connexion...")
        time.sleep(15)  # Attendre la connexion
        print(" Connexion réussie")
        return True
        
    except Exception as e:
        print(f" Erreur lors de la connexion: {e}")
        return False

def navigate_to_profile(driver, profile_url):
    """Navigue vers un profil Facebook spécifique"""
    try:
        driver.get(profile_url)
        time.sleep(4)
        print(f" Navigation vers le profil: {profile_url}")
        return True
    except Exception as e:
        print(f" Erreur lors de la navigation: {e}")
        return False

In [ ]:
# Fonctions utilitaires pour les profils
def extract_profile_name(profile_url):
    """Extrait le nom du profil depuis l'URL"""
    try:
        if '/profile.php?id=' in profile_url:
            # Pour les profils avec ID numérique
            profile_id = profile_url.split('id=')[1].split('&')[0]
            return f"profile_{profile_id}"
        else:
            # Pour les profils avec nom personnalisé
            profile_name = profile_url.rstrip('/').split('/')[-1]
            return profile_name.replace('.', '_')
    except:
        return f"profile_{int(time.time())}"  # Fallback avec timestamp

def add_profile_urls(new_urls):
    """Ajoute de nouvelles URLs à la liste globale"""
    global PROFILE_URLS
    
    if isinstance(new_urls, str):
        new_urls = [new_urls]
    
    for url in new_urls:
        if url not in PROFILE_URLS:
            PROFILE_URLS.append(url)
            print(f" URL ajoutée: {url}")
        else:
            print(f" URL déjà présente: {url}")
    
    print(f" Total URLs dans la liste: {len(PROFILE_URLS)}")
    return PROFILE_URLS

def show_current_urls():
    """Affiche la liste actuelle des URLs"""
    print(f" URLs configurées ({len(PROFILE_URLS)}):")
    for i, url in enumerate(PROFILE_URLS, 1):
        profile_name = extract_profile_name(url)
        print(f"  {i}. {profile_name} - {url}")
    return PROFILE_URLS

def remove_url(url_or_index):
    """Supprime une URL par son adresse ou son index"""
    global PROFILE_URLS
    
    try:
        if isinstance(url_or_index, int):
            # Suppression par index
            if 0 <= url_or_index < len(PROFILE_URLS):
                removed_url = PROFILE_URLS.pop(url_or_index)
                print(f" URL supprimée: {removed_url}")
            else:
                print(f" Index invalide: {url_or_index}")
        else:
            # Suppression par URL
            if url_or_index in PROFILE_URLS:
                PROFILE_URLS.remove(url_or_index)
                print(f" URL supprimée: {url_or_index}")
            else:
                print(f" URL non trouvée: {url_or_index}")
    except Exception as e:
        print(f" Erreur lors de la suppression: {e}")
    
    return PROFILE_URLS

def clear_all_urls():
    """Vide la liste des URLs"""
    global PROFILE_URLS
    PROFILE_URLS.clear()
    print(" Liste des URLs vidée")
    return PROFILE_URLS

In [ ]:
# Fonctions d'extraction des données
def extract_posts_from_page(driver, profile_url=None):
    """Extrait les données des posts de la page actuelle"""
    try:
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")
        posts_data = []
        
        # Extraire le nom du profil pour l'identifier
        profile_name = extract_profile_name(profile_url) if profile_url else "unknown_profile"
        
        posts = soup.find_all("div", {"class": "x1n2onr6 x1ja2u2z"})
        
        for post in posts:
            try:
                # Extraction du texte du post (titre)
                message_elements = post.find_all("div", {"data-ad-preview": "message"})
                post_text = " ".join([msg.get_text(strip=True) for msg in message_elements])
                
                # Extraction des commentaires
                comments = []
                comment_elements = post.find_all("div", {"class": "xdj266r x14z9mp xat24cr x1lziwak x1vvkbs"})
                for comment in comment_elements:
                    comment_text = comment.get_text(strip=True)
                    if comment_text and len(comment_text) > 20:  # Filtrer les textes trop courts
                        comments.append(comment_text)
                
                if post_text:  # Seulement ajouter si il y a du contenu
                    posts_data.append({
                        "profile_name": profile_name,
                        "profile_url": profile_url,
                        "post_text": post_text,
                        "comments": " | ".join(comments),  # Séparer les commentaires par des |
                        "extracted_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    })
            except Exception as e:
                print(f" Erreur lors de l'extraction d'un post: {e}")
                continue
                
        print(f" {len(posts_data)} posts extraits de cette page ({profile_name})")
        return posts_data
        
    except Exception as e:
        print(f" Erreur lors de l'extraction des posts: {e}")
        return []

def remove_duplicates(data_list):
    """Supprime les posts en double"""
    seen = set()
    unique_data = []
    for data in data_list:
        # Utiliser le texte du post comme identifiant unique
        identifier = data.get('post_text', '')
        if identifier and identifier not in seen:
            seen.add(identifier)
            unique_data.append(data)
    
    print(f" {len(data_list) - len(unique_data)} doublons supprimés")
    return unique_data

In [ ]:
# Fonctions de scraping
def scrape_facebook_posts(driver, max_posts=10, profile_url=DEFAULT_PROFILE_URL):
    """Fonction principale de scraping des posts Facebook pour un profil"""
    profile_name = extract_profile_name(profile_url)
    print(f" Début du scraping de {profile_name} - Objectif: {max_posts} posts")
    
    # Navigation vers le profil
    if not navigate_to_profile(driver, profile_url):
        return []
    
    all_posts = []
    scroll_attempts = 0
    max_scroll_attempts = 50  # Limite de sécurité
    
    while len(all_posts) < max_posts and scroll_attempts < max_scroll_attempts:
        # Extraire les posts de la page actuelle
        posts = extract_posts_from_page(driver, profile_url)
        all_posts.extend(posts)
        
        # Supprimer les doublons
        all_posts = remove_duplicates(all_posts)
        
        print(f" Progression {profile_name}: {len(all_posts)}/{max_posts} posts uniques collectés")
        
        # Si on a assez de posts, arrêter
        if len(all_posts) >= max_posts:
            break
        
        # Faire défiler pour charger plus de contenu
        slow_scroll(driver)
        scroll_attempts += 1
        
        # Attendre un peu pour que le nouveau contenu se charge
        time.sleep(random.uniform(2, 4))
    
    # Limiter au nombre demandé
    final_posts = all_posts[:max_posts]
    print(f" Scraping de {profile_name} terminé: {len(final_posts)} posts collectés")
    
    return final_posts

def scrape_multiple_profiles(driver, profile_urls, max_posts_per_profile=10):
    """Scrape plusieurs profils Facebook"""
    all_profiles_data = []
    
    print(f" Scraping de {len(profile_urls)} profils avec {max_posts_per_profile} posts chacun")
    
    for i, url in enumerate(profile_urls, 1):
        print(f"\n{'='*60}")
        print(f" Scraping du profil {i}/{len(profile_urls)}: {url}")
        print(f"{'='*60}")
        
        try:
            posts_data = scrape_facebook_posts(driver, max_posts_per_profile, url)
            
            if posts_data:
                all_profiles_data.extend(posts_data)
                print(f" Profil {i} terminé: {len(posts_data)} posts collectés")
            else:
                print(f" Aucun post collecté pour le profil {i}")
            
            # Pause entre les profils pour éviter la détection
            if i < len(profile_urls):  # Pas de pause après le dernier profil
                wait_time = random.uniform(5, 10)
                print(f" Pause de {wait_time:.1f}s avant le prochain profil...")
                time.sleep(wait_time)
                
        except Exception as e:
            print(f" Erreur lors du scraping du profil {i} ({url}): {e}")
            continue
    
    print(f"\n Scraping multi-profils terminé!")
    print(f" Total: {len(all_profiles_data)} posts collectés sur {len(profile_urls)} profils")
    
    return all_profiles_data

In [ ]:
#  Fonctions de sauvegarde
def save_to_csv(posts_data, filename=None):
    """Sauvegarde les données dans un fichier CSV bien formaté"""
    if not posts_data:
        print(" Aucune donnée à sauvegarder")
        return None
    
    # Générer un nom de fichier si non fourni
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d _%H%M%S")
        filename = f"facebook_posts_comments_{timestamp}.csv"
    
    filepath = os.path.join(OUTPUT_FOLDER, filename)
    
    try:
        # Définir les colonnes dans l'ordre souhaité
        fieldnames = ['profile_name', 'profile_url', 'post_text', 'comments', 'extracted_at']
        
        with open(filepath, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            # Écrire l'en-tête
            writer.writeheader()
            
            # Écrire les données
            for post in posts_data:
                writer.writerow(post)
        
        print(f" Données sauvegardées dans: {filepath}")
        return filepath
        
    except Exception as e:
        print(f" Erreur lors de la sauvegarde CSV: {e}")
        return None

def display_posts_summary(posts_data):
    """Affiche un résumé des posts collectés"""
    if not posts_data:
        print(" Aucune donnée à afficher")
        return
    
    print(f"\n RÉSUMÉ DES POSTS COLLECTÉS")
    print(f"{'='*50}")
    print(f"Nombre total de posts: {len(posts_data)}")
    
    # Statistiques par profil
    profiles_stats = {}
    for post in posts_data:
        profile_name = post.get('profile_name', 'unknown')
        if profile_name not in profiles_stats:
            profiles_stats[profile_name] = 0
        profiles_stats[profile_name] += 1
    
    print(f"Nombre de profils scrapés: {len(profiles_stats)}")
    for profile, count in profiles_stats.items():
        print(f"  - {profile}: {count} posts")
    
    print(f"\n APERÇU DES PREMIERS POSTS:")
    print(f"{'='*50}")
    
    for idx, post in enumerate(posts_data[:3], 1):
        print(f"\nPost {idx} ({post.get('profile_name', 'unknown')}):")
        print(f" Profil: {post.get('profile_url', 'N/A')}")
        print(f" Titre: {post['post_text'][:100]}...")
        print(f" Commentaires: {post['comments'][:100]}...")
        print(f"{'-'*30}")

In [ ]:
# Fonction d'exécution complète
def run_complete_scraping(max_posts=10, profile_urls=None):
    """Exécute le scraping complet avec sauvegarde pour un ou plusieurs profils"""
    driver = None
    
    # Utiliser la liste par défaut si aucune URL fournie
    if profile_urls is None:
        profile_urls = PROFILE_URLS
    
    # Convertir en liste si une seule URL est fournie
    if isinstance(profile_urls, str):
        profile_urls = [profile_urls]
    
    print(f" Démarrage du scraping de {len(profile_urls)} profil(s)")
    
    try:
        # Initialiser le driver
        driver = initialize_driver()
        if not driver:
            return None
        
        # Se connecter à Facebook
        if not login_to_facebook(driver):
            return None
        
        # Scraper les posts
        if len(profile_urls) == 1:
            # Scraping d'un seul profil
            posts_data = scrape_facebook_posts(driver, max_posts, profile_urls[0])
        else:
            # Scraping de plusieurs profils
            posts_data = scrape_multiple_profiles(driver, profile_urls, max_posts)
        
        if not posts_data:
            print(" Aucun post n'a pu être collecté")
            return None
        
        # Afficher le résumé
        display_posts_summary(posts_data)
        
        # Sauvegarder les données dans un seul fichier CSV
        csv_file = save_to_csv(posts_data)
        
        print(f"\n Scraping terminé avec succès!")
        print(f" Fichier sauvegardé: {csv_file}")
        
        return posts_data, csv_file
        
    except Exception as e:
        print(f" Erreur générale: {e}")
        return None
        
    finally:
        close_driver(driver)

In [ ]:




# Exemples d'utilisation
print("\n CODE PRÊT ! Voici comment utiliser le scraper multi-profils :")
print("="*70)

# Exécution
if __name__ == "__main__": 
    
    run_complete_scraping()